# Predicting Language Program from Economic Index 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from   sklearn.linear_model import LogisticRegression

In [2]:
full_data = pd.read_csv("full_data.csv")

In [3]:
display(full_data)

,Borough,Borough/Citywide Office (B/CO),District,DBN,school_name_x,School Category,Program,Language,General/Special Education,school_name_y,...,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty,Economic Need Index,mostCommonEthnicGroup,language_number,ethnicity_number,predicted_language
0,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,General Education,P.S. 020 Anna Silver,...,0.220,61,0.131,352,75.9%,78.2%,% Hispanic,1,2,Spanish
1,Manhattan,Manhattan,1,01M378,School for Global Leaders,Junior High-Intermediate-Middle,Dual Language,Chinese,General Education,School for Global Leaders,...,0.407,17,0.083,172,84.3%,84.7%,% Hispanic,1,2,Spanish
2,Brooklyn,Brooklyn North,14,14K250,P.S. 250 George H. Lindsay,Elementary,Dual Language,Chinese,General Education,P.S. 250 George H. Lindsay,...,0.167,44,0.108,330,81.1%,86.4%,% Hispanic,1,2,Spanish
3,Queens,Queens North,24,24Q013,P.S. 013 Clement C. Moore,Elementary,Dual Language,Chinese,General Education,P.S. 013 Clement C. Moore,...,0.147,385,0.277,1065,76.6%,79.4%,% Hispanic,1,2,Spanish
4,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Spanish,General Education,P.S. 020 Anna Silver,...,0.220,61,0.131,352,75.9%,78.2%,% Hispanic,9,2,Spanish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Brooklyn,Brooklyn North,13,13K020,P.S. 020 Clinton Hill,Elementary,Dual Language,French,General Education,P.S. 020 Clinton Hill,...,0.158,26,0.048,288,53.4%,48.0%,% Black,2,1,Haitian Creole
260,Brooklyn,Brooklyn North,13,13K691,Fort Greene Preparatory Academy,Junior High-Intermediate-Middle,Dual Language,French,General Education,Fort Greene Preparatory Academy,...,0.316,12,0.057,192,91.9%,88.3%,% Black,2,1,Haitian Creole
261,Brooklyn,Brooklyn North,16,16K005,P.S. 005 Dr. Ronald McNair,Elementary,Dual Language,French,General Education,P.S. 005 Dr. Ronald McNair,...,0.258,13,0.052,197,79.4%,81.6%,% Black,2,1,Haitian Creole
262,Manhattan,Manhattan,3,03M242,P.S. 242 - The Young Diplomats Magnet Academy,Elementary,Dual Language,Italian,Special Education,P.S. 242 - The Young Diplomats Magnet Academy,...,0.329,11,0.079,118,84.3%,85.0%,% Black,4,1,Haitian Creole


In [4]:
#cleaning the economic need index 

####### drop all schools where the economic need index of rows is above 95%  
# print(full_data[full_data["Economic Need Index"]=="Above 95%"])
full_data.drop([5,13,18,19,50,51,52,56,60,64,69,70,71,88,119,186,187],inplace=True)
print("this is the number of rows " + str(len(full_data)))

#rename economic need index 
full_data = full_data.rename(columns={"Economic Need Index":"economic_need_index"})

this is the number of rows 247


In [5]:

#remove percent sign
full_data["df_percent"] = full_data["economic_need_index"].str.rstrip('%').astype(float)
full_data.head(10)

,Borough,Borough/Citywide Office (B/CO),District,DBN,school_name_x,School Category,Program,Language,General/Special Education,school_name_y,...,# English Language Learners,% English Language Learners,# Poverty,% Poverty,economic_need_index,mostCommonEthnicGroup,language_number,ethnicity_number,predicted_language,df_percent
0,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,General Education,P.S. 020 Anna Silver,...,61,0.131,352,75.9%,78.2%,% Hispanic,1,2,Spanish,78.2
1,Manhattan,Manhattan,1,01M378,School for Global Leaders,Junior High-Intermediate-Middle,Dual Language,Chinese,General Education,School for Global Leaders,...,17,0.083,172,84.3%,84.7%,% Hispanic,1,2,Spanish,84.7
2,Brooklyn,Brooklyn North,14,14K250,P.S. 250 George H. Lindsay,Elementary,Dual Language,Chinese,General Education,P.S. 250 George H. Lindsay,...,44,0.108,330,81.1%,86.4%,% Hispanic,1,2,Spanish,86.4
3,Queens,Queens North,24,24Q013,P.S. 013 Clement C. Moore,Elementary,Dual Language,Chinese,General Education,P.S. 013 Clement C. Moore,...,385,0.277,1065,76.6%,79.4%,% Hispanic,1,2,Spanish,79.4
4,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Spanish,General Education,P.S. 020 Anna Silver,...,61,0.131,352,75.9%,78.2%,% Hispanic,9,2,Spanish,78.2
6,Manhattan,Bronx,2,02M542,Manhattan Bridges High School,High school,Dual Language,Spanish,General Education,Manhattan Bridges High School,...,189,0.363,443,85.2%,92.7%,% Hispanic,9,2,Spanish,92.7
7,Manhattan,Manhattan,3,03M075,P.S. 075 Emily Dickinson,Elementary,Dual Language,Spanish,General Education,P.S. 075 Emily Dickinson,...,68,0.131,353,67.9%,65.4%,% Hispanic,9,2,Spanish,65.4
8,Manhattan,Manhattan,3,03M075,P.S. 075 Emily Dickinson,Elementary,Dual Language,Spanish,Special Education,P.S. 075 Emily Dickinson,...,68,0.131,353,67.9%,65.4%,% Hispanic,9,2,Spanish,65.4
9,Manhattan,Manhattan,3,03M145,"P.S. 145, The Bloomingdale School",Elementary,Dual Language,Spanish,General Education,"P.S. 145, The Bloomingdale School",...,39,0.099,279,70.6%,77.4%,% Hispanic,9,2,Spanish,77.4
10,Manhattan,Manhattan,3,03M165,P.S. 165 Robert E. Simon,Elementary,Dual Language,Spanish,General Education,P.S. 165 Robert E. Simon,...,37,0.086,269,62.3%,64.0%,% Hispanic,9,2,Spanish,64.0


In [12]:
economic_model = LogisticRegression(multi_class = 'multinomial', solver = 'newton-cg').fit(full_data["df_percent"], full_data["language_number"])
# economic_model.score(X_test, y_test)

ValueError: Expected 2D array, got 1D array instead:
array=[78.2 84.7 86.4 79.4 78.2 92.7 65.4 65.4 77.4 64.  85.2 85.2 91.7 93.6
 87.4 87.4 93.5 93.5 89.6 89.6 94.4 94.3 87.6 91.2 90.2 94.6 86.3 84.7
 66.3 66.3 83.7 90.4 66.3 86.8 88.9 88.9 78.  78.  78.  92.  93.3 91.1
 92.9 93.  67.8 86.7 91.  94.5 94.6 90.7 93.2 94.2 94.8 87.3 93.2 93.9
 92.9 92.4 92.4 94.8 91.8 93.6 93.5 93.2 88.2 91.7 88.2 94.6 89.7 88.5
 81.6 81.6 86.8 86.8 94.9 93.4 93.4 43.4 84.6 87.7 58.1 88.1 88.1 85.7
 69.8 87.8 57.2 88.5 76.6 88.5 88.5 73.8 67.5 88.9 88.9 81.7 88.1 88.1
 77.  85.7 86.4 86.4 86.4 86.7 88.8 92.6 92.3 89.2 88.3 72.9 78.7 79.4
 90.  88.  88.  88.2 69.8 80.6 84.4 82.8 88.9 88.1 88.1 87.6 87.6 90.3
 86.9 84.3 90.  85.2 60.9 76.1 86.5 83.1 87.6 77.4 82.4 81.4 58.  58.8
 89.1 89.1 61.5 52.1 76.6 76.6 83.8 83.7 57.5 82.  91.8 83.9 92.8 80.3
 92.8 75.1 80.5 89.9 83.9 91.1 84.  92.1 90.9 89.1 93.8 93.4 77.9 90.6
 90.6 93.4 91.8 43.4 69.9 88.9 77.4 70.8 70.1 67.2 67.2 81.1 85.1 82.2
 94.8 94.8 93.1 84